## Model Name
MultI-Task  Ordinal  Regression  (MITOR)  
Spatial  In-complete Multi-task Deep leArning (SIMDA)  
Pending Task: Deciding the name

In [ ]:
# Reading Data
import scipy.io
data = scipy.io.loadmat('../data/MITOR_Data.mat') # data format: dict
x_data = data['X'] # shape (26,550,1289) (Task ID (Spatial location), Sample (Temporal-Days), Features(TF-IDF)) # Civil Unrest Data
y_data = data['Y'] # shape (26, 550)
adj_data = data['adj'] # shape (26, 26)

In [ ]:
# Parameters
# learning rate for gradient descent
lr=0.8
# L2,1 norm co-effecient (can be chose using validation set)
alpha=0
# threshold theta constriants co-efficient (can be chose using validation set)
beta=0.01

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
y_data_3d = np.expand_dims(y_data, axis=2)
data_concat = np.swapaxes(np.concatenate((x_data, y_data_3d),axis = 2),0,1)
data_train, data_test = train_test_split(data_concat, test_size=0.5, random_state=42, shuffle=True)  # Split the set

In [ ]:
data_train.shape

In [ ]:
# X_train = torch.FloatTensor(data_train[:,:,0:-1])
# Y_train = torch.FloatTensor(data_train[:,:,-1])
# X_test = torch.FloatTensor(data_test[:,:,0:-1])
# Y_test = torch.FloatTensor(data_test[:,:,-1])
import torch
X_train = torch.swapaxes(torch.FloatTensor(data_train[:,:,0:-1]), 0, 1) # Shape([26, 275, 1289])
Y_train = torch.swapaxes(torch.FloatTensor(data_train[:,:,-1]), 0, 1)
X_test = torch.swapaxes(torch.FloatTensor(data_test[:,:,0:-1]), 0, 1)
Y_test = torch.swapaxes(torch.FloatTensor(data_test[:,:,-1]), 0, 1)


In [ ]:
X_train.shape

In [ ]:
import torch
import torch.nn.functional as F
class extract_hidden(torch.nn.Module):
        def __init__(self, input_size, hidden_size):
            super(MITADOR, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        def forward(self, x):
            hidden = self.fc1(x)
            return hidden # h(x)
# loss = use the equation.
# Loss. backward.
fc1 = torch.nn.Linear(1289, 10)
# X_hidden = torch.swapaxes(fc1(X_train), 0, 1)

X_hidden = F.relu(fc1(X_train))

In [ ]:
values = Y_train.unique()
k = values.shape[0]
TRAIN_size = Y_train.shape[1]
TASK_num = Y_train.shape[0]
BIG = float('inf')
SMALL = float('-inf')
# d=demension of feature vector
d=X_hidden.shape[-1]

R = torch.zeros((TASK_num, TASK_num))
for i in range(TASK_num):
    nb=sum(adj_data[:,1]==1)
    if nb>0:
        R[:,i]=-torch.from_numpy(adj_data[:,i]/nb)
        R[i,i]=1
L=R*np.transpose(R)
E=torch.eye((TASK_num))
# initialize parameters for training data for each task(size= d*TASK_num)
W=torch.nn.Parameter(torch.empty(d, TASK_num).normal_(mean=0,std=0.1))
U=W
Y1=torch.zeros(W.shape)
# initialize threshold size : (k-1)*TASK_num
pi=torch.nn.Parameter(torch.ones(k-1,1)*1/k)
# assign each class with same probability
cum_pi = torch.cumsum(pi, dim=0) # Need to check here
T=torch.log(cum_pi/ (1-cum_pi))
T = T.repeat(1, TASK_num)
Y2 = torch.zeros(T.shape)
V=T
Y3=torch.zeros(k-2,T.shape[1])
# initialize learning rate lambda
lambda_w=lr
lambda_t=lr
# admm param
rho=1

# set max number of iterations for GD on W,T  update (can be shorter if meet stop critira)
Max_iteration=15
f1inv=torch.linalg.inv(beta*L+2*rho*E)
f2inv=torch.linalg.inv(beta*L+rho*E)

In [ ]:
# X_hidden.shape
# for i in range (x_data.shape[0]):
#     for j in range(x_data.shape[1]):
#         print(x_data[i,j,:])
        
#     X_Hidden = fc1(X_train) # X_train (Whole train set (S, t, features)) # 275 # use some non-linearity, start with sigmoid

In [ ]:

# ADI
    #Max-iteration
        #TASK-i
            # Train_size-j
            X_Hidden = fc1(x_train[i,j,:]) # X_train (just one sample) -> x_hidden (1,1, 10)
            if-else
            Loss.backward
            optimizer.step()

In [ ]:
# l_cum=-torch.log(sig1-sig2) # threshold is wrong, sig1 should be greater than sig2
sig1

In [ ]:
for kkkkkkk in range(6):
    print(kkkkkkk)

In [ ]:
pi.shape[0,:]

In [ ]:
# for adi= 1: 500
adi = 1
L_old = float('inf')
I = torch.zeros(Max_iteration)
L_train = torch.zeros(Max_iteration)

In [ ]:
# Update W,T via gradient decent
# for t = 1 : Max_iteration
t=1
# cross tasks overall loss
loss=0;
GW=torch.zeros((d,TASK_num))
GT=torch.zeros((k-1,TASK_num))

In [ ]:
logsig = torch.nn.LogSigmoid()
for i in range(TASK_num):
        for j in range(TRAIN_size):
            # calculate what is the model predict is
            X_hidden = F.relu(fc1(X_train[i,j,:]))
            if Y_train[i,j]==k:
                sig1=1
                sig2=logsig(T[int(Y_train[i,j])-2,i] - torch.matmul(X_hidden,W[:,i]))
                dif=SMALL
            elif Y_train[i,j]==1:
                sig1=logsig(T[int(Y_train[i,j])-1,i] - torch.matmul(X_hidden,W[:,i]))
                sig2=0
                dif=SMALL
            else:
                sig1= logsig(T[int(Y_train[i,j])-1,i] - torch.matmul(X_hidden,W[:,i]))
                sig2= logsig(T[int(Y_train[i,j])-2,i] - torch.matmul(X_hidden,W[:,i]))
                dif= T[int(Y_train[i,j])-2,i]-T[int(Y_train[i,j])-1,i]
            # loss
            l_cum=-torch.log(sig1-sig2)

            if(l_cum==float('inf')):
                print("error: inf loss, please decrease learning rate!")
            loss=loss+l_cum
            # we want to minimize p_cum w.r.t W and T
            # Gradient Decent
            gradient_w = X_hidden*(1-sig1-sig2)
            GW[:,i]=GW[:,i]+gradient_w
            gradient_t=torch.zeros(k-1)
            # Form canonical vector e_yi and e_yi-1
            if Y_train[i,j]==k:
                gradient_t[int(Y_train[i,j])-2]=sig2
            elif Y_train[i,j]==1:
                gradient_t[int(Y_train[i,j])-1]=sig1-1
            else:
                gradient_t[int(Y_train[i,j])-1]=sig1-1+1/(1-torch.exp(-dif))
                gradient_t[int(Y_train[i,j])-2]=sig2-1+1/(1-torch.exp(dif))
            GT[:,i]=GT[:,i]+gradient_t
        #end j
        #end i

In [ ]:
GW=1/TRAIN_size*1/TASK_num*GW+Y1+rho*(W-U)
GT=1/TRAIN_size*1/TASK_num*GT+Y2+rho*(T-V)
# update parameters
W=W-lambda_w*GW
T=T-lambda_t*GT
I[t]=t
L_train[t]=1/TRAIN_size*1/TASK_num*loss+torch.sum(torch.sum(Y1*(W-U)))+(rho / 2.) * torch.sum(torch.sum(torch.square((W-U))))+torch.sum(torch.sum(Y2*(T-V)))+(rho / 2) * torch.sum(torch.sum(torch.square(T-V)))

# if L_old-L_train[t]<1e-2:
#     break
L_old = L_train[t] 
# end t

In [ ]:
# Update U
# https://github.com/KentonMurray/ProxGradPytorch
# pip install proximal-gradient
# It is not a closed form solution. So, it is not exactly the matlab solution. 
# When it is called gradient then it is not analytical solution or closed form solution.
# https://pypi.org/project/pyproximal/
# python operator for proximal operator. or covert Matlab code to python.
# https://github.com/PyLops/pyproximal
U=prox_l21(W+1/rho*Y1, alpha/rho);
import proximal_gradient.proximalGradient as pg
U_old=U
U = W+1/rho*Y1
pg.l21(U, reg=alpha/rho)

In [ ]:
U = W+1/rho*Y1
alpha=0.1
sdsd=alpha/rho

In [ ]:
U[:,1].detach().numpy().shape

In [ ]:
import pyproximal
prox_l21 = pyproximal.L21(ndim=1, sigma = 1)
# x=np.arange(-1, 1, 0.1)
x=np.ones((5,5))
tau = 0.5
xp = prox_l21.prox(U[:,1].detach().numpy(), alpha/rho)

In [ ]:
x.shape

In [ ]:
# Update V
V_old=V
V[1,:]=1/rho*Y2[1,:]+T[1,:]
for i in range(1,k-1): # Not sure about the range starting point
    f1=torch.matmul((torch.matmul(beta*V[i-1,:],L)+Y2[i,:]+rho*(T[i,:]+V[i-1,:])+Y3[i-1,:]),f1inv)
    f2=torch.matmul((torch.matmul(beta*V[i-1,:],L)+Y2[i,:]+rho*T[i,:]+Y3[i-1,:]),f2inv)
    V[i,f1<=V[i-1,:]]=f1[f1<=V[i-1,:]]
    V[i,f2>V[i-1,:]]=f2[f2>V[i-1,:]]

In [ ]:
# Update Y
Y1=Y1+rho*(W-U)
Y2=Y2+rho*(T-V)
for i in range(1,k-1):# Not sure about the range starting point
    Y3[i-1,:]=torch.maximum(Y3[i-1,:]+rho*(V[i-1,:]-V[i,:]),torch.zeros(TASK_num))

In [ ]:
# Compute ADMM residuals and update rho correspondingly
s=rho*torch.sqrt(torch.sum(torch.sum(torch.square(U_old-U))))+rho*torch.sqrt(torch.sum(torch.sum(torch.square(V_old-V))))
r=torch.sqrt(torch.sum(torch.sum(torch.square(U-W))))+torch.sqrt(torch.sum(torch.sum(torch.square(V-T))))
if(r>10*s):
    rho = 2*rho
#   update inverse matrix for computation of V-update when rho changes
    f1inv=torch.linalg.inv(beta*L+2*rho*E)
    f2inv=torch.linalg.inv(beta*L+rho*E)
else:
    if(10*r<s):
        rho = rho/2
#       update inverse matrix for computation of V-update when rho changes
        f1inv=torch.linalg.inv(beta*L+2*rho*E)
        f2inv=torch.linalg.inv(beta*L+rho*E)

In [ ]:
print('ADMM iteration: {}, r-residual: {}, s-residual: {}, rho: {}\n'.format(adi,r,s,rho))
    
# Check for stop criteria
# if (r<1e-2 && s<1e-2)
#     break
# end
# end adi

In [ ]:
T.shape

In [ ]:
# The final learned model parameter set
B=torch.cat((T, -W),0);

In [ ]:
# More Reference
# https://github.com/bzantium/pytorch-admm-pruning/blob/master/main.py

In [ ]:
import configure as config
import model
# Reading Data
import scipy.io
import numpy as np
from sklearn.model_selection import train_test_split
import torch

data = scipy.io.loadmat('../data/MITOR_Data.mat') # data format: dict
x_data = data['X'] # shape (26,550,1289) (Task ID (Spatial location), Sample (Temporal-Days), Features(TF-IDF)) # Civil Unrest Data
y_data = data['Y'] # shape (26, 550)
adj_data = data['adj'] # shape (26, 26)

y_data_3d = np.expand_dims(y_data, axis=2)
data_concat = np.swapaxes(np.concatenate((x_data, y_data_3d),axis = 2),0,1)
data_train, data_test = train_test_split(data_concat, test_size=0.5, random_state=42, shuffle=True)  # Split the set

X_train = torch.swapaxes(torch.FloatTensor(data_train[:,:,0:-1]), 0, 1) # Shape([26, 275, 1289])
Y_train = torch.swapaxes(torch.FloatTensor(data_train[:,:,-1]), 0, 1)
X_test = torch.swapaxes(torch.FloatTensor(data_test[:,:,0:-1]), 0, 1)
Y_test = torch.swapaxes(torch.FloatTensor(data_test[:,:,-1]), 0, 1)

MITOR_V2 = model.Model(config.input_dim, config.hidden_dim)
trained_param = MITOR_V2.train_model(X_train,Y_train,adj_data,config.lr, config.alpha, config.beta)

In [ ]:
# predict
TEST_size = Y_test.shape[1]
TASK_num = Y_test.shape[0]
# Test learned model on test set
MZE=torch.zeros(TASK_num,1)
# mean aboslute error rate
MAE=torch.zeros(TASK_num,1)
label=torch.zeros(TEST_size,TASK_num)

In [ ]:
# https://fa.bianp.net/blog/2013/logistic-ordinal-regression/
# https://github.com/fabianp/minirank/blob/master/minirank/logistic.py
def ordinal_logistic_predict(w, theta, X):
    """
    Parameters
    ----------
    w : coefficients obtained by ordinal_logistic
    theta : thresholds
    """
    unique_theta = np.sort(np.unique(theta))
    out = X.dot(w)
    unique_theta[-1] = np.inf # p(y <= max_level) = 1
    tmp = out[:, None].repeat(unique_theta.size, axis=1)
    return np.argmax(tmp < unique_theta, axis=1)

In [ ]:
# B=-W, T # Check + and negative both.
# unpack it by task. with different W, T based on different task
trained_param.shape
W.shape
T.shape
# X_test.shape

In [ ]:
trained_param[:,1].shape

In [ ]:
def predict(X_test, Y_test, trained_param):
    [T,W] = trained_param
    TASK_num = Y_test.shape[0]
    for i in range(TASK_num):
        X_hidden = F.relu(fc1(X_test[i,:,:]))
        unique_theta = torch.sort(torch.unique(T[:,i]))
        # unique_theta.values[-1]=torch.inf
        out = torch.matmul(X_hidden,W[:,i])
        tmp = out[:, None].repeat(1,unique_theta.values.shape[0])
        pred = torch.argmax((tmp < unique_theta.values).long(), dim=1)
        if i==0:
            Y_pred = pred.unsqueeze(0)
        else:
            Y_pred = torch.cat((Y_pred,pred.unsqueeze(0)),0)

    zero_one_sum = 0
    for i in range(TASK_num):
        zero_one_task = metrics.zero_one_loss(Y_test[0,:], Y_pred[0,:])
        zero_one_sum+=zero_one_task
    mze = zero_one_sum/TASK_num
    mae = (Y_pred != Y_test).long().sum()/Y_test.ravel().shape[0]
    
    return mze, mae


In [ ]:
((Y_pred != Y_test).long().sum()/Y_test.ravel().shape[0]).item()

In [ ]:
error = torch.abs(Y_pred - Y_test).sum().data
squared_error = ((Y_pred - Y_test)*(Y_pred - Y_test)).sum().data
# runnning_mae += error
# runnning_mse += squared_error

In [ ]:
(Y_pred - Y_test).shape

In [ ]:
import pickle 
model_dump_file = open('../saved_model/MITOR_V2_trained.obj', 'wb') 
pickle.dump(MITOR_V2, model_file)
# file_pi2 = open('MITOR_V2_trained.obj', 'r') 
# object_pi2 = pickle.load(file_pi2)

In [ ]:
model_load_file = open('../saved_model/MITOR_V2_trained.obj', 'rb')
MITOR_V2_trained = pickle.load(model_load_file)

In [ ]:
Y_test.ravel().shape[0]

In [ ]:
from sklearn import metrics
metrics.mean_absolute_error(Y_test, Y_pred)


In [ ]:
metrics.mean_absolute_error(Y_test[0,:], Y_pred[0,:])

In [ ]:
# out[:, None].shape
# X_hidden.shape # 275,10
# out.shape # 275
# tmp.shape
# X_test.shape # 26, 75, 1289
# W[:,i].shape
# unique_theta.values.shape[0]
torch.argmax((tmp < unique_theta.values).long(), dim=0)

In [ ]:
tmp.shape

In [ ]:
unique_theta[-1]

In [ ]:
import configure as config
import model
# Reading Data
import scipy.io
import numpy as np
from sklearn.model_selection import train_test_split
import torch

data = scipy.io.loadmat('../data/MITOR_Data.mat') # data format: dict
x_data = data['X'] # shape (26,550,1289) (Task ID (Spatial location), Sample (Temporal-Days), Features(TF-IDF)) # Civil Unrest Data
y_data = data['Y'] # shape (26, 550)
adj_data = data['adj'] # shape (26, 26)

y_data_3d = np.expand_dims(y_data, axis=2)
data_concat = np.swapaxes(np.concatenate((x_data, y_data_3d),axis = 2),0,1)
data_train, data_test = train_test_split(data_concat, test_size=0.5, random_state=42, shuffle=True)  # Split the set

X_train = torch.swapaxes(torch.FloatTensor(data_train[:,:,0:-1]), 0, 1) # Shape([26, 275, 1289])
Y_train = torch.swapaxes(torch.FloatTensor(data_train[:,:,-1]), 0, 1)
X_test = torch.swapaxes(torch.FloatTensor(data_test[:,:,0:-1]), 0, 1)
Y_test = torch.swapaxes(torch.FloatTensor(data_test[:,:,-1]), 0, 1)

MITOR_V2 = model.Model(config.input_dim, config.hidden_dim)
print("Training Model")
trained_param = MITOR_V2.fit(X_train,Y_train,adj_data,config.lr, config.alpha, config.beta)
# print("----------------------------------------------------------")
# print("Testing Model")
# [mze, mae] = MITOR_V2.predict(X_test, Y_test, trained_param)

In [ ]:
# importing pandas as pd 
import pandas as pd 
   
# list of name, degree, score
adi_list = []
r_list = []
s_list = []
rho_list = []
mze_list = []
mae_list = []

   
# dictionary of lists 
dict = {'name': nme, 'degree': deg, 'score': scr} 
     
df = pd.DataFrame(dict)
  
# saving the dataframe
df.to_csv(r'C:\Users\Admin\Desktop\file3.csv', index=False)

In [ ]:
import pickle

som = SOM_CLASS()
fileObject = <any file-like object>
pickle.dump(som, fileObject)
#...
som = pickle.load(fileObject)
som.work()

In [ ]:
fc1.state_dict()['weight']

In [ ]:
fc1.state_dict().keys()

In [ ]:
fc1.bias.data

In [ ]:
lines = ['model name: {}'.format(2),
             'data: {}'.format(3),
             'w_from_matlab: {}'.format('ddfgh')]
line2=['momentum: {}'.format('dd'), 'kkkk: {}'.format('jj')]
lines.append(line2)
print(lines)

In [ ]:
MITOR_V2_trained.mlp_model.fc1.weight

In [1]:
import configure as config
import utils
import model
# Reading Data
import scipy.io
import pickle
print("Testing Model")

data = scipy.io.loadmat(config.data_location)  # data format: dict
[X_train, Y_train, X_test, Y_test, adj_data] = utils.data_formulate(data)
print('Data Loaded')
model_name = '23_05_2022_10_27_35_MITOR_V2_mlp.obj'
model_location = '../saved_model/' + model_name
print("Loading Model from {}".format(model_location))

model_load_file = open(model_location, 'rb')
MITOR_V2_trained = pickle.load(model_load_file)
# # MITOR_V2_trained.to(device)
[mze, mae, Y_pred] = MITOR_V2_trained.predict(X_test, Y_test, return_pred=True)
test_str = "On Test Data: Mean Zero-one Error = {}, Mean Absolute Error = {}".format(mze, mae)
print(test_str)
Y_pred_dict = {'Y_pred': Y_pred.detach().cpu().numpy()}
Y_test_dict = {'Y_test': Y_test.detach().cpu().numpy()}
scipy.io.savemat('../data/debug_data/y_pred_flu12wihY_MLP.mat', Y_pred_dict)
scipy.io.savemat('../data/debug_data/y_test_flu12wihY_MLP.mat', Y_test_dict)

Testing Model


C:\ProgramData\Anaconda3\envs\Incomplete_label_multi_task\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


Data Loaded
Loading Model from ../saved_model/23_05_2022_10_27_35_MITOR_V2_mlp.obj
On Test Data: Mean Zero-one Error = 0.8314363956451416, Mean Absolute Error = 1.170918345451355


In [3]:
MITOR_V2_trained.mlp_model.fc1.weight

Parameter containing:
tensor([[ 3.1221e-03,  9.4547e-04,  7.5977e-04,  ...,  6.9691e-04,
          2.4694e-03,  7.0787e-05],
        [-5.2080e-03, -2.9895e-03, -6.6231e-04,  ..., -2.6786e-03,
         -2.0818e-03, -1.1383e-03],
        [-1.5489e-03,  2.0293e-04,  8.5451e-04,  ..., -1.5515e-03,
         -3.0287e-05, -6.5535e-04],
        ...,
        [ 2.6489e-04,  4.6299e-04,  6.6816e-04,  ..., -6.9081e-04,
         -3.0044e-04,  2.0506e-03],
        [-9.1983e-04, -8.7969e-05, -1.0676e-03,  ..., -6.1627e-04,
         -1.3163e-04, -3.0559e-04],
        [ 7.8528e-04,  9.2117e-04, -6.7705e-04,  ...,  7.2534e-04,
          1.1668e-04,  9.9386e-06]], requires_grad=True)

In [4]:
MITOR_V2_trained.W

tensor([[-0.1677,  0.0138,  0.0387,  ...,  0.1395,  0.0127,  0.0325],
        [ 0.1390, -0.0879,  0.0971,  ...,  0.1126, -0.0092, -0.1678],
        [-0.1568,  0.1559,  0.2053,  ...,  0.1459,  0.1819,  0.0912],
        ...,
        [-0.1973, -0.0107,  0.2057,  ..., -0.0867, -0.0482, -0.0486],
        [ 0.1584, -0.0153, -0.0430,  ...,  0.1119,  0.0782,  0.1640],
        [-0.0573, -0.0748, -0.1009,  ..., -0.1592,  0.0640,  0.0086]])

In [ ]:
import scipy.io
import utils
import scipy.io
import torch

# Reading Data
# _sample
B_location = '../data/debug_data//B.mat'
Y_test_location = '../data/debug_data//Y_test.mat'
X_test_location = '../data/debug_data//X_test.mat'

B_mat = scipy.io.loadmat(B_location)
B = torch.tensor(B_mat['B']).float()

Y_test_mat = scipy.io.loadmat(Y_test_location)
Y_test_m = torch.tensor(Y_test_mat['Y_test']).float()

X_test_mat = scipy.io.loadmat(X_test_location)
X_test_m = torch.tensor(X_test_mat['X_test']).float()

[mze, mae, Y_pred] = utils.predict_from_mat(B, X_test_m, Y_test_m)
print('On test set: Mean zero-one Error = {} Mean Absolute Error = {}'.format(mze,mae))
Y_pred_dict = {'Y_pred': Y_pred.detach().cpu().numpy()}
Y_test_dict = {'Y_test': Y_test_m.detach().cpu().numpy()}
scipy.io.savemat('../data/debug_data/y_pred_flu_12_usingMatB.mat' , Y_pred_dict)
scipy.io.savemat('../data/debug_data/y_test_flu_12_usingMatB.mat' , Y_test_dict)

In [ ]:
Y_pred.unique()